# Creating and Visualizing DEMs from LIDAR points

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

In this notebook we will:
* Create high-quality DEMs from LiDAR point clouds and compute topographic parameters
* Create webmap visualizations


## 1. Startup

Import Python standard library and IPython packages we need. Start GRASS session in Nags Head project.

In [5]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

import grass.script as gs
import grass.jupyter as gj

gj.init("./nags_head/PERMANENT");

## 2. Initial Point Cloud Analysis

In [13]:
lidar_files = sorted(Path('./Jockeys_Ridge_LiDAR/').glob('*.txt'))
lidar_files

[PosixPath('Jockeys_Ridge_LiDAR/JR_19961016.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_19971002.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_19980907.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_19990909.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_19990918.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_19991104.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_2001.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_20040925.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_20051126.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_2007.txt'),
 PosixPath('Jockeys_Ridge_LiDAR/JR_20080327.txt')]

TODO: switch to pandas or something nicer than using strings to make a table...

In [ ]:
gs.run_command("g.region", raster="naip_2014")
    
report = "date\tres\tn\trange\n"

resolutions=[ 0.5, 2, 5, 10 ]

for f in files:
    report += f + "\n"
    for res in resolutions:
        report += "\t" + str(res) + "\t"

        # Set the resolution.
        gs.run_command("g.region", res=res)
        
        # Get the per cell count.
        gs.run_command(
            "r.in.xyz",
            input=f,
            output="JR_stats_n",
            method="n",
            fs=",",
            overwrite=True
        )
        gs.run_command(
            "r.null",
            map="JR_stats_n",
            setnull=0
        )
        
        # Get the mean per cell count.
        stats = gs.parse_command("r.univar", flags="ge", map="JR_stats_n")
        report += str(stats["mean"]) + "\t"
        
        # Get the per cell range.
        gs.run_command(
            "r.in.xyz",
            input=f,
            output="JR_stats_range",
            method="range",
            fs=",",
            overwrite=True
        )
        gs.run_command(
            "r.mapcalc",
            expression="JR_stats_range_c=if(isnull(JR_stats_n), null(), JR_stats_range)",
            overwrite=True
        )
        
        # Get the mean per cell range.
        stats = gs.parse_command(
            "r.univar",
            flags="ge",
            map="JR_stats_range_c"
        )
        report += str(stats["mean"]) + "\n"
print(report)

## 3. Creating DEMs

### Mask low density areas

### Binning vs Spline

TODO: Do we need the systematic error correction part?